# **Maven Super Agent**

## Setup

In [6]:
import os
from IPython.display import Markdown, display
# Setting environment variables
os.environ['OPENAI_API_KEY'] = ''


In [31]:
research_topic = "AVX512"

In [8]:
sources = ["pdf",
           "wikipedia",
           "arxiv",
           "youtube",
           "webpage"]

## PDF Reader

In [10]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
documents_pdf = SimpleDirectoryReader("data/local_pdfs").load_data()

In [11]:
# PDF unit testing
index_pdf = VectorStoreIndex.from_documents(documents_pdf)
index_pdf.storage_context.persist(persist_dir="pdf_reader")
query_engine_pdf = index_pdf.as_query_engine()
response = query_engine_pdf.query("What is SIMD width of avx 512?")
display(Markdown(f"{response}"))

The SIMD width of AVX-512 is 512 bits.

## Wikipedia Reader

In [12]:
# wikipedia pages
from llama_index.readers.wikipedia import WikipediaReader

documents_wiki = WikipediaReader().load_data(
    pages=[research_topic]
)

In [13]:
# Optimized Wiki reader

import wikipedia
from llama_index.readers.wikipedia import WikipediaReader
from llama_index.core import SimpleDirectoryReader
import os

def clean_wikipedia_text(page_text):
    lines = page_text.split('\n')
    cleaned_lines = []
    in_reference_section = False
    
    for line in lines:
        # Skip lines that are part of the references section
        if "References" in line:
            in_reference_section = True
        if in_reference_section:
            continue
        
        # Skip lines that are part of the contents or media
        if line.strip().lower().startswith(('contents', 'references', 'external links', 'see also', 'notes', 'further reading')):
            continue
        
        # Skip lines that are empty
        if not line.strip():
            continue
        
        cleaned_lines.append(line)
    
    return '\n'.join(cleaned_lines)

def fetch_clean_wikipedia_page(topic):
    try:
        page = wikipedia.page(topic)
        cleaned_text = clean_wikipedia_text(page.content)
        return cleaned_text, page.title
    except wikipedia.exceptions.PageError:
        print(f"Wikipedia page for topic '{topic}' does not exist.")
        return None, None
    except wikipedia.exceptions.DisambiguationError as e:
        print(f"Disambiguation error for topic '{topic}'. Options are: {e.options}")
        return None, None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None, None

research_topic = "AVX-512"
cleaned_page_text, page_title = fetch_clean_wikipedia_page(research_topic)

if cleaned_page_text:
    # Define the directory and file name
    output_dir = "./wiki_docs"
    os.makedirs(output_dir, exist_ok=True)  # Ensure the directory exists
    output_file_path = os.path.join(output_dir, f"{research_topic}.txt")
    
    # Write the cleaned text to a file
    with open(output_file_path, "w", encoding="utf-8") as file:
        file.write(cleaned_page_text)
    
    # Load the file using SimpleDirectoryReader
    reader = SimpleDirectoryReader(input_dir=output_dir)
    documents_wiki = reader.load_data()
    
else:
    print(f"Could not retrieve or clean Wikipedia page for topic '{research_topic}'.")


In [14]:
# Wikipedia unit testing
from llama_index.core import VectorStoreIndex
index_wikipedia = VectorStoreIndex.from_documents(documents_wiki)
index_wikipedia.storage_context.persist(persist_dir="wiki_reader")
query_engine_wiki = index_wikipedia.as_query_engine()
response = query_engine_wiki.query("What is a spicy quote on avx-512?")
display(Markdown(f"{response}"))

Linus Torvalds expressed his strong opinion on AVX-512, stating that he hopes it "dies a painful death" and criticized Intel for focusing on creating benchmark-boosting instructions rather than addressing real issues.

In [15]:
from llama_index.tools.wikipedia.base import WikipediaToolSpec
wiki_spec = WikipediaToolSpec()
wiki_spec.search_data(research_topic)

'AVX-512 are 512-bit extensions to the 256-bit Advanced Vector Extensions SIMD instructions for x86 instruction set architecture (ISA) proposed by Intel in July 2013, and first implemented in the 2016 Intel Xeon Phi x200 (Knights Landing), and then later in a number of AMD and other Intel CPUs (see list below). AVX-512 consists of multiple extensions that may be implemented independently. This policy is a departure from the historical requirement of implementing the entire instruction block. Only the core extension AVX-512F (AVX-512 Foundation) is required by all AVX-512 implementations.\nBesides widening most 256-bit instructions, the extensions introduce various new operations, such as new data conversions, scatter operations, and permutations. The number of AVX registers is increased from 16 to 32, and eight new "mask registers" are added, which allow for variable selection and blending of the results of instructions. In CPUs with the vector length (VL) extension—included in most AV

## Arxiv Reader

In [32]:
from llama_index.readers.papers import ArxivReader
import time

num_papers_read = 3
loader = ArxivReader()

retry_attempts = 3
for attempt in range(retry_attempts):
    try:
        documents_arxiv = loader.load_data(
            search_query=research_topic, max_results=num_papers_read
        )
        # If successful, break out of the loop
        break
    except ValueError as e:
        if 'No files found in .papers' in str(e):
            print(f"Attempt {attempt + 1} failed: {e}. Retrying...")
        else:
            # Handle other ValueError exceptions
            print(f"An unexpected error occurred: {e}")
            break
else:
    # If all attempts fail, handle the error gracefully
    print(f"Failed to load papers for topic '{research_topic}' after {retry_attempts} attempts.")
    documents_arxiv = None


In [33]:
# Arxiv unit test
index_arxiv = VectorStoreIndex.from_documents(documents_arxiv)
index_arxiv.storage_context.persist(persist_dir="arxiv_reader")
query_engine_arxiv = index_arxiv.as_query_engine()
response = query_engine_arxiv.query("What is the difference between avx512 and avx2?")
display(Markdown(f"{response}"))

AVX512 introduces new features like AVX-512BW (Byte and Word) that are not supported in AVX2. AVX512 also offers wider registers and additional instructions compared to AVX2.

In [34]:
from llama_index.tools.arxiv.base import ArxivToolSpec
arxiv_tool = ArxivToolSpec()
arxiv_tool.arxiv_query(research_topic)

[Document(id_='113d4f09-7b55-4d6c-bbb8-041c8c09b84a', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={}, text='http://arxiv.org/pdf/1909.02871v1: Galois Field Arithmetics for Linear Network Coding using AVX512 Instruction Set Extensions\nLinear network coding requires arithmetic operations over Galois fields, more\nspecifically over finite extension fields. While coding over GF(2) reduces to\nsimple XOR operations, this field is less preferred for practical applications\nof random linear network coding due to high chances of linear dependencies and\ntherefore redundant coded packets. Coding over larger fields such as GF(16) and\nGF(256) does not have that issue, but is significantly slower. SIMD vector\nextensions of processors such as AVX2 on x86-based systems or NEON on ARM-based\ndevices offer the potential to increase performance by orders of magnitude.\n  In this paper we present an implementation of different algorithms 

## Youtube reader

In [35]:
from llama_index.readers.youtube_transcript import YoutubeTranscriptReader
from llama_index.core import VectorStoreIndex

import json
from datetime import datetime, timedelta

number_of_videos = 3

def filter_and_sort_videos(json_data, max_duration_minutes=20, max_publish_time_years=1):
    # Load the JSON data
    data = json.loads(json_data)
    
    # Function to convert duration from "MM:SS" or "HH:MM:SS" to seconds
    def duration_to_seconds(duration):
        parts = list(map(int, duration.split(":")))
        if len(parts) == 2:
            return parts[0] * 60 + parts[1]
        elif len(parts) == 3:
            return parts[0] * 3600 + parts[1] * 60 + parts[2]
        return 0
    
    # Filter videos based on duration and publish time
    filtered_videos = []
    max_duration_seconds = max_duration_minutes * 60
    one_year_ago = datetime.now() - timedelta(days=max_publish_time_years * 365)
    
    for video in data["videos"]:
        duration_seconds = duration_to_seconds(video["duration"])
        
        publish_time_parts = video["publish_time"].split()
        try:
            publish_time_num = int(publish_time_parts[0])
        except ValueError:
            # Skip the video if the publish time is not a number
            continue
        
        publish_time_unit = publish_time_parts[1]
        publish_delta = timedelta(days=0)
        
        if "month" in publish_time_unit:
            publish_delta = timedelta(days=publish_time_num * 30)
        elif "year" in publish_time_unit:
            publish_delta = timedelta(days=publish_time_num * 365)
        
        publish_date = datetime.now() - publish_delta
        
        if duration_seconds <= max_duration_seconds and publish_date >= one_year_ago:
            filtered_videos.append(video)
    
    # Sort videos by views
    sorted_videos = sorted(filtered_videos, key=lambda x: int(x["views"].replace(" views", "").replace(",", "")), reverse=True)
    
    # Get the full URLs of the top n videos
    top_n_urls = ["https://www.youtube.com/watch?v=" + video["id"] for video in sorted_videos[:number_of_videos]]
    
    return top_n_urls

from youtube_search import YoutubeSearch

research_topic = "AVX-512"
json_data = YoutubeSearch(research_topic, max_results=10).to_json()
yt_links = filter_and_sort_videos(json_data)

print(yt_links)  # For debugging, ensure URLs are correct
documents_youtube = YoutubeTranscriptReader().load_data(ytlinks=yt_links)


['https://www.youtube.com/watch?v=bskEGP0r3hE', 'https://www.youtube.com/watch?v=P1s6ZQcDvUs', 'https://www.youtube.com/watch?v=olwiOyUkrGY']


In [36]:
# YouTube unit test
index_youtube = VectorStoreIndex.from_documents(documents_youtube)
index_youtube.storage_context.persist(persist_dir="youtube_reader")
query_engine_youtube = index_youtube.as_query_engine()
response = query_engine_youtube.query("Whats Ian's take on avx 512?")
display(Markdown(f"{response}"))

Ian's take on AVX 512 is that it has been a bit lackluster in terms of communication from Intel's side, making it confusing for the average consumer. There have been critiques regarding the thermal impact of AVX 512 instructions on CPUs, causing clock speed decreases and performance issues. However, Ian also mentions that AMD saw a performance improvement in frequency with their AVX 512 implementation by splitting the 512 Ops at the instruction execution unit level. Overall, Ian suggests that Intel could improve AVX 512 by implementing a minimal die area AVX 512 implementation to make it more accessible and beneficial for code writers.

## Webpage

In [37]:
from llama_index.tools.duckduckgo import DuckDuckGoSearchToolSpec
from llama_index.readers.web import SimpleWebPageReader
search_tool = DuckDuckGoSearchToolSpec()
full_search = search_tool.duckduckgo_full_search(research_topic, max_results=3)
urls = [article['href'] for article in full_search]
documents_webpage = SimpleWebPageReader(html_to_text=True).load_data(
    urls
)

In [38]:
# Webpage unit test
from llama_index.core import SummaryIndex
from IPython.display import Markdown, display

index_webpage = SummaryIndex.from_documents(documents_webpage)
index_webpage.storage_context.persist(persist_dir="webpage_reader")
query_engine_webpage = index_webpage.as_query_engine()
response = query_engine_webpage.query("What are the downsides of using avx512?")
display(Markdown(f"{response}"))

The downsides of using AVX-512 include potential challenges related to software optimization, as AVX-512 instructions require specific compiler support to fully leverage their capabilities. Additionally, AVX-512 instructions can consume more power and generate more heat, which may be a concern for systems with limited cooling capabilities. Furthermore, not all applications may benefit significantly from AVX-512 instructions, leading to potential inefficiencies in utilizing these advanced vector extensions.

## Multi document agent

In [39]:
from llama_index.core import (
    VectorStoreIndex,
    SimpleKeywordTableIndex,
    SimpleDirectoryReader,
)
from llama_index.core import SummaryIndex
from llama_index.core.schema import IndexNode
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.llms.openai import OpenAI
from llama_index.core.callbacks import CallbackManager
from llama_index.agent.openai import OpenAIAgent
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.node_parser import SentenceSplitter
import os
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.llm = OpenAI(temperature=0, model="gpt-3.5-turbo")
Settings.embed_model = OpenAIEmbedding(model="text-embedding-ada-002")

node_parser = SentenceSplitter()

# Build agents dictionary
agents = {}
query_engines = {}

# this is for the baseline
all_nodes = []

source_dict ={}
source_dict['pdf'] = documents_pdf
source_dict['arxiv'] = documents_arxiv
source_dict['youtube'] = documents_youtube
source_dict['wikipedia'] = documents_wiki
source_dict['webpage'] = documents_webpage

In [40]:

# Iterate over the combined documents
for idx, source in enumerate(sources):
    nodes = node_parser.get_nodes_from_documents(source_dict[source])
    all_nodes.extend(nodes)

    # Check if a directory for the wiki title exists, if not, create it and build a new vector index
    if not os.path.exists(f"./data/{source}"):
        vector_index = VectorStoreIndex(nodes)  # Create a vector index from the nodes
        vector_index.storage_context.persist(persist_dir=f"./data/{source}")  # Persist the vector index to disk
    else:
        # If directory exists, load the existing vector index from storage
        vector_index = load_index_from_storage(
            StorageContext.from_defaults(persist_dir=f"./data/{source}"),
        )

    # Create a summary index from the nodes
    summary_index = SummaryIndex(nodes)

    # Define query engines for vector and summary indexes
    vector_query_engine = vector_index.as_query_engine(llm=Settings.llm)
    summary_query_engine = summary_index.as_query_engine(llm=Settings.llm)

    # Define tools for querying the vector and summary indexes
    query_engine_tools = [
        QueryEngineTool(
            query_engine=vector_query_engine,
            metadata=ToolMetadata(
                name="vector_tool",
                description=(
                    f"Useful for questions related to specific aspects of {source}.\n\n"
                    "You are a research assistant providing accurate information from local PDFs, Wikipedia, YouTube, webpages, and arXiv papers. Follow these guidelines:\n"
                    "1. For broad overviews, historical context, and general information, use Wikipedia.\n"
                    "2. For in-depth research and technical details, use arXiv papers.\n"
                    "3. For the latest news and updates, use webpages.\n"
                    "4. For tutorials, demonstrations, reviews and feedback from the industry, use YouTube.\n"
                    "5. For specific, pre-verified documents, use local PDFs.\n"
                    "General Instructions:\n"
                    "Choose the source that best fits the query. Combine information from multiple sources when necessary. Ensure the information is accurate and relevant.\n\n"
                    "You must ALWAYS use at least one of the provided tools when answering a question; do NOT rely on prior knowledge alone."
                ),
            ),
        ),
        QueryEngineTool(
            query_engine=summary_query_engine,
            metadata=ToolMetadata(
                name="summary_tool",
                description=(
                    f"Useful for questions related to specific aspects of {source}.\n\n"
                    "You are a research assistant providing accurate information from local PDFs, Wikipedia, YouTube, webpages, and arXiv papers. Follow these guidelines:\n"
                    "1. For broad overviews, historical context, and general information, use Wikipedia.\n"
                    "2. For in-depth research and technical details, use arXiv papers.\n"
                    "3. For the latest news and updates, use webpages.\n"
                    "4. For tutorials, demonstrations, reviews and feedback from the industry, use YouTube.\n"
                    "5. For specific, pre-verified documents, use local PDFs.\n"
                    "General Instructions:\n"
                    "Choose the source that best fits the query. Combine information from multiple sources when necessary. Ensure the information is accurate and relevant.\n\n"
                    "You must ALWAYS use at least one of the provided tools when answering a question; do NOT rely on prior knowledge alone."
                ),
            ),
        ),
    ]

    # Build an OpenAI agent with the defined tools and a custom system prompt
    function_llm = OpenAI(model="gpt-4")  # Initialize the LLM with GPT-4 model
    agent = OpenAIAgent.from_tools(
        query_engine_tools,
        llm=function_llm,
        verbose=True,
        system_prompt=f"""\
You are a specialized agent designed to answer queries about {research_topic}.
You must ALWAYS use at least one of the tools provided when answering a question; do NOT rely on prior knowledge.\
""",
    )

    # Store the agent and the query engine in their respective dictionaries
    agents[source] = agent
    query_engines[source] = vector_index.as_query_engine(similarity_top_k=2)
    

In [41]:
all_tools = []
for source in sources:
    tool_summary = (
        f"Useful for questions related to specific aspects of {source}.\n\n"
        "You are a research assistant providing accurate information from local PDFs, Wikipedia, YouTube, webpages, and arXiv papers. Follow these guidelines:\n"
        "1. For broad overviews, historical context, and general information, use Wikipedia.\n"
        "2. For in-depth research and technical details, use arXiv papers.\n"
        "3. For the latest news and updates, use webpages.\n"
        "4. For tutorials, demonstrations, reviews and feedback from the industry, use YouTube.\n"
        "5. For specific, pre-verified documents, use local PDFs.\n"
        "General Instructions:\n"
        "Choose the source that best fits the query. Combine information from multiple sources when necessary. Ensure the information is accurate and relevant.\n\n"
        "You must ALWAYS use at least one of the provided tools when answering a question; do NOT rely on prior knowledge alone."
    )
    
    doc_tool = QueryEngineTool(
        query_engine=agents[source],
        metadata=ToolMetadata(
            name=f"tool_{source}",
            description=tool_summary,
        ),
    )
    all_tools.append(doc_tool)


In [42]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [43]:
from llama_index.agent.openai import OpenAIAgent

top_agent = OpenAIAgent.from_tools(
    tool_retriever=obj_index.as_retriever(similarity_top_k=3),
    system_prompt=""" \
        f"Useful for questions related to specific aspects of {source}.\n\n"
        "You are a research assistant providing accurate information from local PDFs, Wikipedia, YouTube, webpages, and arXiv papers. Follow these guidelines:\n"
        "1. For broad overviews, historical context, and general information, use Wikipedia.\n"
        "2. For in-depth research and technical details, use arXiv papers.\n"
        "3. For the latest news and updates, use webpages.\n"
        "4. For tutorials, demonstrations, reviews and feedback from the industry, use YouTube.\n"
        "5. For specific, pre-verified documents, use local PDFs.\n"
        "General Instructions:\n"
        "Choose the source that best fits the query. Combine information from multiple sources when necessary. Ensure the information is accurate and relevant.\n\n"
        "You must ALWAYS use at least one of the provided tools when answering a question; do NOT rely on prior knowledge alone."
""",
    verbose=True,
)

In [44]:
base_index = VectorStoreIndex(all_nodes)
base_query_engine = base_index.as_query_engine(similarity_top_k=4)

In [45]:
# Historical context should use wikipedia
response = top_agent.query(f"What was used before the invention of {research_topic}")
display(Markdown(f"{response}"))

Added user message to memory: What was used before the invention of AVX-512
=== Calling Function ===
Calling function: tool_arxiv with args: {"input":"AVX-512"}
Added user message to memory: AVX-512
=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "AVX-512"
}
Got output: AVX-512 is an instruction set architecture extension introduced by Intel that supports 512-bit SIMD instructions. It allows for parallel processing of multiple data elements in a single operation, enhancing performance for tasks like sparse matrix/vector multiplication. AVX-512 provides a wide range of vector instructions that can significantly accelerate computations on compatible processors.

Got output: AVX-512, or Advanced Vector Extensions 512, is an extension of the x86 instruction set architecture (ISA) introduced by Intel. It supports 512-bit SIMD (Single Instruction, Multiple Data) instructions, which means it can process multiple data elements in parallel within a single operati

AVX-512 is an extension of the x86 instruction set architecture introduced by Intel. Before the invention of AVX-512, other SIMD (Single Instruction, Multiple Data) instruction sets were used to enhance performance for certain tasks. Some of the SIMD instruction sets used before the invention of AVX-512 include:

1. SSE (Streaming SIMD Extensions): SSE introduced 128-bit SIMD instructions to the x86 architecture, providing a significant performance boost for multimedia and gaming applications.

2. AVX (Advanced Vector Extensions): AVX extended the SIMD capabilities to 256 bits, allowing for even more parallel processing of data elements.

3. AVX2: AVX2 further extended the SIMD capabilities to 256 bits and introduced new instructions to improve performance for integer and floating-point operations.

These SIMD instruction sets were used before the introduction of AVX-512 to enhance parallel processing and improve performance in various applications.

In [46]:
# Technical details should refer to arxiv
response = top_agent.query(f"Give me the technical details behind {research_topic}")
display(Markdown(f"{response}"))

Added user message to memory: Give me the technical details behind AVX-512
=== Calling Function ===
Calling function: tool_arxiv with args: {"input":"AVX-512"}
Added user message to memory: AVX-512
=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "AVX-512"
}
Got output: AVX-512 is an instruction set architecture extension introduced by Intel that supports 512-bit SIMD instructions. It allows for parallel processing of multiple data elements in a single operation, enhancing performance for tasks like sparse matrix/vector multiplication. AVX-512 provides a wide range of vector instructions to accelerate computations on x86-based processors.

Got output: AVX-512, or Advanced Vector Extensions 512, is an instruction set architecture extension introduced by Intel. It supports 512-bit SIMD (Single Instruction, Multiple Data) instructions, which means it can process multiple data elements in parallel within a single operation. This capability significantly enhan

AVX-512 is an instruction set architecture extension introduced by Intel that supports 512-bit SIMD instructions, allowing for parallel processing of multiple data elements within a single operation. It enhances performance for tasks like sparse matrix/vector multiplication and provides a wide range of vector instructions to accelerate computations on x86-based processors.

In [112]:
# For reviews and and feed back use youtube
response = top_agent.query(f"What is the review or feedback from the industry on {research_topic}")
display(Markdown(f"{response}"))

Added user message to memory: What is the review or feedback from the industry on AVX512
=== Calling Function ===
Calling function: tool_youtube with args: {"input":"AVX512 review"}
Added user message to memory: AVX512 review
=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "AVX512 review"
}
Got output: AVX512 is not mentioned in the provided context information.

=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "AVX512 review"
}
Got output: AVX512 is a set of advanced instructions designed for demanding computational tasks, particularly in the realm of scientific computing, artificial intelligence, and data analytics. It offers enhanced performance by allowing for parallel processing of a larger number of data elements compared to previous instruction sets.

Got output: AVX512 is a set of advanced instructions designed for demanding computational tasks, particularly in the realm of scientific computing, artificial intelligen

AVX512 is a set of advanced instructions designed for demanding computational tasks, particularly in scientific computing, artificial intelligence, and data analytics. It offers enhanced performance by allowing for parallel processing of a larger number of data elements compared to previous instruction sets. For more detailed reviews and feedback, you might want to check out specific industry forums or technical blogs.

In [115]:
# Latest news shuld use webpages/ wikipedia
response = top_agent.query(f"What is the latest news and updates on {research_topic}")
display(Markdown(f"{response}"))

Added user message to memory: What is the latest news and updates on AVX512
=== Calling Function ===
Calling function: tool_wikipedia with args: {"input":"AVX512"}
Added user message to memory: AVX512
=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "AVX512"
}
Got output: AVX512 is not mentioned in the provided context information.

=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "AVX512"
}
Got output: AVX512 is a set of advanced instructions for processors that can significantly accelerate performance for certain types of computations.

Got output: AVX512, or Advanced Vector Extensions 512, is a set of instructions for processors that can significantly enhance performance for specific types of computations. These instructions allow for the simultaneous processing of multiple data points, which can be particularly beneficial in fields such as scientific computing, machine learning, and multimedia applications.



AVX512, or Advanced Vector Extensions 512, is a set of instructions for processors that can significantly enhance performance for specific types of computations. These instructions allow for the simultaneous processing of multiple data points, which can be particularly beneficial in fields such as scientific computing, machine learning, and multimedia applications.

In [116]:
# Latest news shuld use webpages/ wikipedia
response = top_agent.query(f"What are the registers used in {research_topic}")
display(Markdown(f"{response}"))

Added user message to memory: What are the registers used in AVX512
=== Calling Function ===
Calling function: tool_wikipedia with args: {"input":"AVX512 registers"}
Added user message to memory: AVX512 registers
=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "AVX512 registers"
}
Got output: Sundar Pichai's career and background do not relate to AVX512 registers.

=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "AVX512 registers"
}
Got output: Sundar Pichai's background and career details are provided in the context.

=== Calling Function ===
Calling function: summary_tool with args: {
  "input": "AVX512 registers"
}
Got output: Sundar Pichai's career and background do not relate to AVX512 registers.

=== Calling Function ===
Calling function: vector_tool with args: {
  "input": "AVX512 registers"
}
Got output: Sundar Pichai's background and career information do not include any details related to AVX512 registers.

=== Ca

AVX-512 registers are 512 bits wide and can hold up to 64 bytes of data. They are used to enhance performance for certain types of computational tasks, such as vector and matrix operations.